E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [ ]:
import torch

words = open('names.txt','r').read().splitlines()

In [65]:
# EX1

# Construct data set

chars = sorted(list(set(''.join(words))))
stoi_chars = {s:i+1 for i,s in enumerate(chars)}
stoi_chars['.'] = 0
itos_chars = {i:s for s,i in stoi.items()}


char_pairs = []

for ch in chars:
    char_pairs.append('.' + ch)

for ch1 in chars:
    for ch2 in chars:
        char_pairs.append(ch1+ch2)

for ch in chars:
    char_pairs.append(ch + '.')

stoi_pairs = {s: i for i,s in enumerate(char_pairs)}
itos_pairs = {i:s for s, i in stoi_pairs.items()}


xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi_pairs[ch1 + ch2]
        ix2 = stoi_chars[ch3]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [66]:
xs

tensor([  4, 142, 350,  ..., 700, 675, 699])

In [102]:
# one_hot encoding for input to network (size of 728 for 728 possible char pairs)
xenc = torch.nn.functional.one_hot(xs, num_classes=728).float()

# Randomly Init NN
W = torch.randn((728, 27), requires_grad=True)

In [108]:
for i in range(1000):
    # forward pass
    logits = xenc @ W

    # Normalize output to prob dist w softmax
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)

    # We want to maximize the probability of the correct ys, given xs as input
    loss = -probs[torch.arange(len(ys)), ys].log().mean()

    # backward pass

    W.grad = None
    loss.backward()
    W.data += -5 * W.grad

    print(loss.item())

2.085571527481079
2.0855705738067627
2.0855696201324463
2.085568904876709
2.0855679512023926
2.0855672359466553
2.085566282272339
2.0855655670166016
2.0855648517608643
2.085563898086548
2.0855631828308105
2.0855624675750732
2.085561513900757
2.0855607986450195
2.085559844970703
2.085559129714966
2.0855581760406494
2.085557222366333
2.0855562686920166
2.0855555534362793
2.085554838180542
2.0855538845062256
2.085552930831909
2.085552453994751
2.0855515003204346
2.085550546646118
2.085549831390381
2.0855491161346436
2.085548162460327
2.08554744720459
2.0855464935302734
2.085545539855957
2.085545063018799
2.0855441093444824
2.085543155670166
2.0855424404144287
2.0855414867401123
2.085540771484375
2.0855400562286377
2.0855391025543213
2.085538387298584
2.0855374336242676
2.0855367183685303
2.085535764694214
2.0855348110198975
2.0855343341827393
2.085533380508423
2.0855326652526855
2.085531711578369
2.085530996322632
2.0855300426483154
2.085529088973999
2.0855283737182617
2.0855276584625244


In [117]:
for i in range(5):
    out = []
    curr_context = "." + chars[torch.randint(0, 26, (1,)).item()] 
    out.append(curr_context[1])
    
    while True:
        ix = stoi_pairs[curr_context]
        
        xenc = torch.nn.functional.one_hot(torch.tensor([ix]), num_classes=728).float()
        logits = xenc @ W
        probs = torch.softmax(logits, dim=1)
        
        next_ch_ix = torch.multinomial(probs, num_samples=1).item()
        next_ch = itos_chars[next_ch_ix]
        
        if next_ch == '.':
            break
            
        out.append(next_ch)
        
        curr_context = curr_context[1] + next_ch
        
    print(''.join(out))

mazzen
oser
ugurwwaque
wremandracobysonnee
giscaranas
